# Summary
This document corresponds to Exercise 8 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

If [PEPit](https://pypi.org/project/PEPit/) is not already installed, please execute the following cell.

In [ ]:
!pip install pepit

### Exercise 8.1

Compute a worst-case performance for the subgradient method with stepsizes $\{\gamma_k\}_{0\leqslant k\leqslant N}$.

It computes the worst-case $\min_{0\leqslant i\leqslant N}\{f(x_i)-f_\star\}$ when $f$ is convex and $M$-Lipschitz and $\|x_0-x_\star\|^2\leqslant R^2$ for some $x_\star\in\mathrm{argmin}_x \,f(x)$ (and $f_\star=f(x_\star)$).

In [ ]:
from math import sqrt
from PEPit import PEP
from PEPit.functions import ConvexLipschitzFunction

def wc_subgradient_method(M, R, n, gamma, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a convex lipschitz function
    func = problem.declare_function(ConvexLipschitzFunction, M=M)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and xs
    problem.set_initial_condition((x0 - xs)**2 <= R**2)

    # Run n steps of the subgradient method
    x = x0
    gx, fx = func.oracle(x)

    for i in range(n):
        problem.set_performance_metric(fx - fs)
        x = x - gamma[i] * gx
        gx, fx = func.oracle(x)

    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)
    
    return pepit_tau

### Exercise 8.2

Evaluate the output for a few stepsize rules and compare it to the standard guarantee:

In [ ]:
import numpy as np

M = 1
R = 1
n = 5

gamma = [ 1/np.sqrt(i+1) for i in range(n)] 

wc_subgradient_method(M, R, n, gamma, verbose=1)

### Exercise 8.3

Adapt the code above for computing guarantees for the last iterate.

In [ ]:
from math import sqrt
from PEPit import PEP
from PEPit.functions import ConvexLipschitzFunction

def wc_subgradient_method_last(M, R, n, gamma, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a convex lipschitz function
    func = problem.declare_function(ConvexLipschitzFunction, M=M)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and xs
    problem.set_initial_condition((x0 - xs)**2 <= R**2)

    # Run n steps of the subgradient method
    x = x0
    gx, fx = func.oracle(x)

    for i in range(n):
        x = x - gamma[i] * gx
        gx, fx = func.oracle(x)

    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)
    
    return pepit_tau

Test for a few values:

In [ ]:

M = 1
R = 1
n = 7

gamma = [ 1/np.sqrt(n+1) for i in range(n)] 

wc_best = wc_subgradient_method(M, R, n, gamma, verbose=1)
wc_last = wc_subgradient_method_last(M, R, n, gamma, verbose=1)

### Exercise 8.4

Same questions for the quasi-monotone subgradient method.

In [ ]:
def wc_QMsubgradient_method_last(M, R, n, gamma, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a convex lipschitz function
    func = problem.declare_function(ConvexLipschitzFunction, M=M)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and xs
    problem.set_initial_condition((x0 - xs)**2 <= R**2)

    # Run n steps of the subgradient method
    x = x0
    gx, fx = func.oracle(x)
    gx_list = list()
    gx_list.append(gx)
    for i in range(n):
        y = (i+1)/(i+2) * x + 1/(i+2) * x0
        d = 1/(i+2) * np.sum(gx_list)
        x = y - R/M/np.sqrt(n+1) * d
        gx, fx = func.oracle(x)
        gx_list.append(gx)

    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)
    
    return pepit_tau
def wc_QMsubgradient_method_best(M, R, n, gamma, verbose=1):

    # Instantiate PEP
    problem = PEP()

    # Declare a convex lipschitz function
    func = problem.declare_function(ConvexLipschitzFunction, M=M)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and xs
    problem.set_initial_condition((x0 - xs)**2 <= R**2)

    # Run n steps of the subgradient method
    x = x0
    gx, fx = func.oracle(x)
    gx_list = list()
    gx_list.append(gx)
    for i in range(n):
        problem.set_performance_metric(fx - fs)
        y = (i+1)/(i+2) * x + 1/(i+2) * x0
        d = 1/(i+2) * np.sum(gx_list)
        x = y - R/M/np.sqrt(n+1) * d
        gx, fx = func.oracle(x)
        gx_list.append(gx)

    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)
    
    return pepit_tau

Test a few values:

In [ ]:
wc_QMsubgradient_method_last(M, R, n, gamma, verbose=1)
wc_QMsubgradient_method_best(M, R, n, gamma, verbose=1)